In [3]:
import json
import re
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.ticker as mticker

script_dir = os.getcwd() 

#setting up the input file path
script_dir = os.getcwd()
#rel_path = "../data/lacounty_covid.json"
rel_path = "community_mapped_case_data.json"
abs_file_path = os.path.join(script_dir, rel_path)
#setting up the out file path
script_dir = os.getcwd()
out_path_log = "../plots/daily_top6communities_log.png" #this line creates a file with log scale in y axis
abs_out_file_path_log_scale = os.path.join(script_dir, out_path_log)
out_path = "../plots/daily_top6communities.png" #this line creates a file with plot in a regular scale
abs_out_file_path = os.path.join(script_dir, out_path)


class community:
	def __init__(self,name,actual_name,Today_date):
		self.name = name
		self.actual_name = actual_name # for displaying part of figures
		self.dic_confirmed = {}
		self.total_confirmed_so_far = 0
		self.Today_date = Today_date
		self.dic_confirmed_cummulative = np.zeros(len(range(16,self.Today_date)),dtype=int) # for cumulative confirmed cases for each day
		
		
	def calculate_total_confirmed_so_far(self):
		for day in self.dic_confirmed.keys():
			self.total_confirmed_so_far += self.dic_confirmed[day]
	# for adding new entry for each community on every day 		
	def addnumber(self,day, number):
		self.dic_confirmed[day] = number
		# increase cumulative confirmed cases here
		if day == 16:
			self.dic_confirmed_cummulative[day - 16] = number	
		else:
			self.dic_confirmed_cummulative[day - 16] =  self.dic_confirmed_cummulative[day - 16 - 1] +  number	
			print(self.dic_confirmed_cummulative[day - 16])
	# for printing purposes only		
	def print_stat(self):
		print("name", self.name)
		for day in self.dic_confirmed.keys():
			print(day, self.dic_confirmed[day])
	# return the confirmed cases (either daily or cumulative) for each community		
	def plot_info(self,type_plot):
		output = np.zeros(len(range(16,self.Today_date)),dtype=int)
		for index,i in enumerate(list(range(16,self.Today_date))):
			# for daily
			if type_plot == 'daily':
				if i in self.dic_confirmed.keys():
					output[index] =  self.dic_confirmed[i]
				else:
					output[index] = 0
			# for cumulative
			else:
				output = self.dic_confirmed_cummulative			
		return output	

def main(top_i_comm, type_plot, Today_date):
    dict_county = {}  # dictionary of all community objects
    list_communities = []  # list of all community objects

    # Make sure the JSON file is in the correct directory
    with open(abs_file_path) as json_file:
        data = json.load(json_file)
        print(data.keys())
        for day in sorted([int(k) for k in data.keys()]):
            for i in range(len(data[str(day)])):
                actual_name_of_community = data[str(day)][i][0].strip()
                name_of_community = data[str(day)][i][0].strip().lower().replace(' ', '')

                # Clean city names
                prefixes = ['cityof', 'losangeles-', 'unincorporated-']
                for word in prefixes:
                    name_of_community = name_of_community.replace(word, '')

                # Clean confirmed number
                confirmed_cases = data[str(day)][i][0].strip().lower(), re.sub("[^0-9]", "", data[str(day)][i][1].strip())

                if name_of_community != "florence-firestone":
                    print(name_of_community)
                    if name_of_community not in dict_county:
                        dict_county[name_of_community] = community(name_of_community, actual_name_of_community, Today_date)
                        list_communities.append(dict_county[name_of_community])
                        dict_county[name_of_community].addnumber(day, int(confirmed_cases[1]))
                    else:
                        print(day, int(confirmed_cases[1]))
                        dict_county[name_of_community].addnumber(day, int(confirmed_cases[1]))

        # Calculate total confirmed cases
        for community_name in dict_county:
            dict_county[community_name].calculate_total_confirmed_so_far()

        days = list(range(1, Today_date - 16 + 1))

        # Sort and plot top communities
        newlist = sorted(list_communities, key=lambda x: x.total_confirmed_so_far, reverse=True)
        for comm_obj in newlist:
            if comm_obj.name not in ['-investigatedcases', '-underinvestigation', 'longbeach', 'santamonicamountains'] and top_i_comm > 0:
                plt.plot(days, comm_obj.plot_info(type_plot), 'o--', label=comm_obj.actual_name)
                top_i_comm -= 1

        plt.legend()
        plt.xlabel('Days since March 16, 2020')
        plt.ylabel('Cases')
        plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(20))
        plt.grid(True)
        plt.title("Cases reported for top 6 communities")
        plt.savefig(abs_out_file_path)
        plt.yscale('log')
        plt.savefig(abs_out_file_path_log_scale)
        plt.show()

Matplotlib is building the font cache; this may take a moment.
